In [1]:
import datetime as dt
import pandas as pd
import sqlite3

conexion = sqlite3.connect("./piezas.db")
cursor = conexion.cursor()

In [2]:
def consulta_elemento(tabla, nombre):
    query = f"""
        SELECT *
        FROM {tabla}
        WHERE Nombre LIKE '%{nombre}%'
        """
    ID = pd.read_sql(query, conexion).ID

    if len(ID) == 0:
        return 'No se encontró ID.'
    return ID

def consulta_id(tabla, ID:list):
    query = f"""
        SELECT *
        FROM {tabla}
        WHERE ID = '{ID}'
        """
    return pd.read_sql(query, conexion)['Nombre'][0]

def consulta(suministro):
    tabla, algo = suministro[0], suministro[1]

    elemento = input(f'Inserte {algo}:').capitalize()
    resultados = consulta_elemento(tabla, elemento)

    if type(resultados) == str:
        print(resultados)
        return 'salida'
    else:
        if len(resultados) > 1:
            for index, resultado in enumerate(resultados):
                print(index, consulta_id(tabla, resultado))

            index = input(f'Seleccione {algo}:')
            while index.isnumeric() != True or int(index) > len(resultados)-1 or int(len(resultados)-1) <0:
                index = input(f'Inserte un número entre 0-{len(resultados)-1}:')
            ID = resultados[int(index)]
        else:
            ID = resultados[0]
            index = 0
    
    print(consulta_id(tabla, ID))
    
    return str(ID)

def consulta_tabla(tabla):
    return pd.read_sql(f"SELECT * FROM {tabla}", conexion)

In [3]:
def insertar_suministros(fecha, ID_proveedor, ID_pieza, cantidad):
    query = f"INSERT INTO Registro (FECHA, ID_PROVEEDOR, ID_PIEZA, CANTIDAD) VALUES (?,?,?,?)"
    cursor.execute(query,(fecha,ID_proveedor,ID_pieza,cantidad))
    conexion.commit()

    query = """SELECT * FROM Registro"""
    df= pd.read_sql(query, conexion).set_index('FECHA')

    return df

In [4]:
def insertar_campo():
    campo = input("Introduzca 1 para agregar un nuevo proveedor o 2 para agregar una nueva pieza:")
    
    while True:
            
        if campo == "1":

            nombre = input("Introduzca el nombre del nuevo proveedor:")
            direccion = input("Introduzca la dirección, solo el tipo de vía y número:")
            ciudad = input("Introduzca la ciudad:")
            provincia = input("Introduzca la provincia:")
            ID_prov = input("Introduzca un nuevo ID:") # podemos hacer que detecte el indice + 1 del df?? si introducimos un num repetido, esto se rompe al ser el primary key

            query = f"INSERT INTO Proveedores (ID, NOMBRE, DIRECCION, CIUDAD, PROVINCIA) VALUES (?,?,?,?,?)"
            cursor.execute(query,(ID_prov, nombre, direccion, ciudad, provincia))
            conexion.commit()

            query = """SELECT * FROM Proveedores"""
            df= pd.read_sql(query, conexion).set_index('ID')
            break
        
        elif campo == "2":
            nombre = input("Introduzca el nombre de la nueva pieza:")
            color = input("Introduzca el color:")
            precio = float(input("Introduzca el precio:"))
            ID_cat = "unknown"   # unkown ya que no tenemos funcion que consulte categorias
            ID_pieza = input("Introduzca un nuevo ID:") # lo mismo que con el ID proveedor
            
            query = f"INSERT INTO Pieza (ID, NOMBRE, COLOR, PRECIO, ID_CATEGORIA) VALUES (?,?,?,?,?)"
            cursor.execute(query,(ID_pieza, nombre, color, precio, ID_cat))
            conexion.commit()

            query = """SELECT * FROM Pieza"""
            df= pd.read_sql(query, conexion).set_index('ID')
            break

        else:
            campo = input("Introduzca un valor válido entre 1 o 2: 1 - proveedor /  2 - pieza:")

    return df

In [5]:
def insertar_campo(algo):    
    eleccion = input(f'No se encontró {algo}, ¿desea agregar {algo} (S/N)').upper()
    while eleccion != 'S' or eleccion != 'N':
        eleccion = input(f'No se encontró {algo}, ¿desea agregar {algo} (S/N)').upper()
    if eleccion == 'N':
        return

    if algo == "proveedor":

        nombre = input("Introduzca el nombre del nuevo proveedor:")
        direccion = input("Introduzca la dirección, solo el tipo de vía y número:")
        ciudad = input("Introduzca la ciudad:")
        provincia = input("Introduzca la provincia:")
        ID_prov = input("Introduzca un nuevo ID:") # podemos hacer que detecte el indice + 1 del df?? si introducimos un num repetido, esto se rompe al ser el primary key

        query = f"INSERT INTO Proveedores (ID, NOMBRE, DIRECCION, CIUDAD, PROVINCIA) VALUES (?,?,?,?,?)"
        cursor.execute(query,(ID_prov, nombre, direccion, ciudad, provincia))
        conexion.commit()

        query = """SELECT * FROM Proveedores"""
        df= pd.read_sql(query, conexion).set_index('ID')
    
    
    else:
        nombre = input("Introduzca el nombre de la nueva pieza:")
        color = input("Introduzca el color:")
        precio = float(input("Introduzca el precio:"))
        ID_cat = "unknown"   # unkown ya que no tenemos funcion que consulte categorias
        ID_pieza = input("Introduzca un nuevo ID:") # lo mismo que con el ID proveedor
        
        query = f"INSERT INTO Pieza (ID, NOMBRE, COLOR, PRECIO, ID_CATEGORIA) VALUES (?,?,?,?,?)"
        cursor.execute(query,(ID_pieza, nombre, color, precio, ID_cat))
        conexion.commit()

        query = """SELECT * FROM Pieza"""
        df= pd.read_sql(query, conexion).set_index('ID')
        

    return df

In [32]:
insertar_campo()

,Nombre,Color,Precio,ID_Categoria
ID,,,,
1,Tornillo hexagonal,Plateado,0.50,1
2,Tuerca estándar,Plateado,0.30,1
3,Arandela plana,Plateado,0.20,1
4,Resorte helicoidal,Plateado,1.20,2
5,Junta tórica,Negro,0.80,3
6,Correa dentada,Negro,5.99,4
7,Rodamiento de bolas,Gris,8.50,5
8,Engranaje recto,Plateado,3.75,6
9,Eje de transmisión,Negro,12.00,7


In [ ]:
# Está tal cual lo dejamos ayer, cambios en la siguiente celda por si rompo el código

hoy = dt.datetime.now().strftime("%Y-%m-%d")

tablas = [('Proveedores','proveedor'),('Pieza','pieza')]

while True:
    ID_proveedor = consulta(tablas[0])
    if ID_proveedor == 'salida':
        eleccion = input('No se encontró el proveedor, ¿desea agregarlo? (S/N)').upper()
        while eleccion != 'S' or eleccion != 'N':
            eleccion = input('No se encontró el proveedor, ¿desea agregarlo? (S/N)').upper()
        if eleccion == 'N':
            break
        else:
            print('En desarrollo...')
            break

    ID_pieza = consulta(tablas[1])
    if ID_pieza == 'salida':
        eleccion = input('No se encontró la pieza, ¿desea agregarla? (S/N)').upper()
        while eleccion != 'S' or eleccion != 'N':
            eleccion = input('No se encontró la pieza, ¿desea agregarla? (S/N)').upper()
        if eleccion == 'N':
            break
        else:
            print('En desarrollo...')
            break

    cantidad = input(f'Indique cantidad de piezas:')
    while cantidad.isnumeric() != True or int(cantidad) <= 0:
        cantidad = input(f'Inserte una cantidad:')
    
    print(cantidad)

In [6]:
# Copy/Paste de la celda anterior. Intentando integrar la funcion insertar_campo() 

hoy = dt.datetime.now().strftime("%Y-%m-%d")

tablas = [('Proveedores','proveedor'),('Pieza','pieza')]

while True:
    ID_proveedor = consulta(tablas[0])
    if ID_proveedor == 'salida':
        print(insertar_campo(tablas[0][1]))

    ID_pieza = consulta(tablas[1])
    if ID_pieza == 'salida':
        print(insertar_campo(tablas[1][1]))
                
    cantidad = input(f'Indique cantidad de piezas:')
    while cantidad.isnumeric() != True or int(cantidad) <= 0:
        cantidad = input(f'Inserte una cantidad:')
       
        print(cantidad)
    break   

No se encontró ID.


KeyboardInterrupt: Interrupted by user

In [ ]:
insertar_suministros(hoy, ID_proveedor, ID_pieza, cantidad)   # al introducir un nuevo campo, no me lee los IDs del input de insertar_campo()

,ID_PROVEEDOR,ID_PIEZA,CANTIDAD
FECHA,,,
2025-04-24,2,26,15
2025-04-24,100,101,97
2025-04-24,salida,salida,6


In [22]:
consulta_tabla("Pieza")

,ID,Nombre,Color,Precio,ID_Categoria
0,1,Tornillo hexagonal,Plateado,0.50,1
1,2,Tuerca estándar,Plateado,0.30,1
2,3,Arandela plana,Plateado,0.20,1
3,4,Resorte helicoidal,Plateado,1.20,2
4,5,Junta tórica,Negro,0.80,3
5,6,Correa dentada,Negro,5.99,4
6,7,Rodamiento de bolas,Gris,8.50,5
7,8,Engranaje recto,Plateado,3.75,6
8,9,Eje de transmisión,Negro,12.00,7
9,10,Buje de bronce,Dorado,4.25,8


In [ ]:
# Al quitar la tabla del notebook, me ha desaparecido el contenido,la he vuelto a pegar. Tenemos que volcarla en la base de datos.

tabla_suministros = '''
CREATE TABLE Registro(
FECHA    DATE, 
ID_PROVEEDOR    INT,
ID_PIEZA    INT,
CANTIDAD    INT     
)
'''
cursor.execute(tabla_suministros)

query = "SELECT * FROM sqlite_master WHERE type == 'table'"
cursor.execute(query)
cursor.fetchall()

[('table',
  'Pieza',
  'Pieza',
  2,
  'CREATE TABLE Pieza (\n            ID INTEGER PRIMARY KEY AUTOINCREMENT,\n            Nombre TEXT NOT NULL,\n            Color TEXT,\n            Precio REAL,\n            ID_Categoria INTEGER\n        )'),
 ('table',
  'sqlite_sequence',
  'sqlite_sequence',
  3,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'Categoria',
  'Categoria',
  4,
  'CREATE TABLE Categoria (\n            ID INTEGER PRIMARY KEY AUTOINCREMENT,\n            Nombre TEXT NOT NULL UNIQUE\n        )'),
 ('table',
  'Proveedores',
  'Proveedores',
  6,
  'CREATE TABLE Proveedores (\n            ID INTEGER PRIMARY KEY AUTOINCREMENT,\n            Nombre TEXT NOT NULL,\n            Direccion TEXT,\n            Ciudad TEXT,\n            Provincia TEXT,\n            Codigo TEXT UNIQUE,\n            Fecha_Registro DATE DEFAULT CURRENT_DATE\n        )'),
 ('table',
  'Suministros',
  'Suministros',
  8,
  'CREATE TABLE Suministros (\nFECHA    DATE PRIMARY KEY, \nID_PROVEE

In [ ]:
# sintaxis para borrar registros, se puede sacar una función de aquí también

query = '''
DELETE FROM Registro
WHERE ID_PIEZA = "salida" '''

cursor.execute(query)
pd.read_sql("SELECT * FROM Registro", conexion)

,FECHA,ID_PROVEEDOR,ID_PIEZA,CANTIDAD
0,2025-04-24,2,26,15
1,2025-04-24,100,101,97
